In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from math import pi
from datetime import datetime  as dt
import matplotlib
import networkx as nx
import dbase_tools

# Import API key
from api_keys import api_key
single_city_url = "http://api.openweathermap.org/data/2.5/weather?"

## Bokeh imports

In [3]:
#%matplotlib inline
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import DatetimeTickFormatter
##import pandas as pd
#import numpy as np
from bokeh.models.tools import HoverTool
import requests
import time
from math import pi
from datetime import datetime  as dt
import matplotlib
import networkx as nx

In [4]:
output_notebook()

Loading BokehJS ...

In [5]:
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

In [6]:
city_name = "portland"
city_id = "5746545"
city_units = "metric"
covid = "transport_cases_table.csv"

In [7]:
# Create City/location and transport Classes
# ----------------------------------

# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()


# Creates Classes which will serve as the anchor points for our Tables
class State(Base):
    __tablename__ = 'locations'
    id = Column(Integer, primary_key=True)
    date = Column(String(255))
    state_name = Column(String(255))
    fips = Column(Integer)
    cases = Column(Integer)
    deaths = Column(Integer)
    new_cases = Column(Integer)
    new_deaths = Column(Integer)
    country = Column(String(255))
    retail = Column(Integer)
    grocery_and_pharmacy = Column(Integer)
    parks = Column(Integer)
    transit_stations = Column(Integer)
    workplaces = Column(Integer)
    residential = Column(Integer)

In [8]:
engine = create_engine("sqlite:///coviddata.sqlite")
conn = engine.connect()
Base.metadata.create_all(engine)

In [9]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [10]:
covid_df = pd.read_csv(covid)

In [11]:
covid_df

,date,state,fips,cases,deaths,new_cases,new_deaths,country,retail,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,2/15/2020,Washington,53,1,0,0,0,United States,2,-1,8,8,-3,0
1,2/16/2020,Washington,53,1,0,0,0,United States,15,3,19,5,-1,-1
2,2/17/2020,Washington,53,1,0,0,0,United States,12,2,63,-7,-35,6
3,2/18/2020,Washington,53,1,0,0,0,United States,5,2,52,7,-4,0
4,2/19/2020,Washington,53,1,0,0,0,United States,4,3,60,7,-2,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,3/31/2020,West Virginia,54,162,1,17,0,United States,-37,-15,-14,-28,-42,15
1636,4/1/2020,West Virginia,54,191,2,29,1,United States,-34,-10,-14,-26,-43,15
1637,4/2/2020,West Virginia,54,217,2,26,0,United States,-35,-10,-2,-31,-42,16
1638,4/3/2020,West Virginia,54,242,2,25,0,United States,-35,-10,8,-36,-42,17


## data cleaning

In [12]:
covcopy = covid_df.copy()

In [13]:
state_list = covid_df['state'].unique()

In [14]:
state_list

array(['Washington', 'Illinois', 'California', 'Arizona', 'Massachusetts',
       'Wisconsin', 'Texas', 'Nebraska', 'Utah', 'Oregon', 'Florida',
       'New York', 'Rhode Island', 'Georgia', 'New Hampshire',
       'North Carolina', 'New Jersey', 'Colorado', 'Maryland', 'Nevada',
       'Tennessee', 'Hawaii', 'Indiana', 'Kentucky', 'Minnesota',
       'Oklahoma', 'Pennsylvania', 'South Carolina',
       'District of Columbia', 'Kansas', 'Missouri', 'Vermont',
       'Virginia', 'Connecticut', 'Iowa', 'Louisiana', 'Ohio', 'Michigan',
       'South Dakota', 'Arkansas', 'Delaware', 'Mississippi',
       'New Mexico', 'North Dakota', 'Wyoming', 'Alaska', 'Maine',
       'Alabama', 'Idaho', 'Montana', 'West Virginia'], dtype=object)

## Isolate regions by state

In [15]:
all_states = dbase_tools.group_states(covid_df,state_list)

In [16]:
all_states.head()

,date,state,fips,cases,deaths,new_cases,new_deaths,country,retail,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,2/15/2020,Washington,53,1,0,1,0,United States,2,-1,8,8,-3,0
1,2/16/2020,Washington,53,1,0,0,0,United States,15,3,19,5,-1,-1
2,2/17/2020,Washington,53,1,0,0,0,United States,12,2,63,-7,-35,6
3,2/18/2020,Washington,53,1,0,0,0,United States,5,2,52,7,-4,0
4,2/19/2020,Washington,53,1,0,0,0,United States,4,3,60,7,-2,-1


In [17]:
region_list_state = all_states.loc[all_states['state'] == "New York"]

In [18]:
region_list_state.head(5)

,date,state,fips,cases,deaths,new_cases,new_deaths,country,retail,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
510,3/1/2020,New York,36,1,0,1,0,United States,10,5,9,0,3,-1
511,3/2/2020,New York,36,1,0,0,0,United States,6,5,16,-1,4,0
512,3/3/2020,New York,36,2,0,1,0,United States,4,6,6,0,4,0
513,3/4/2020,New York,36,11,0,9,0,United States,7,8,16,0,3,0
514,3/5/2020,New York,36,22,0,11,0,United States,8,9,21,0,4,-1


### start here for state_data_sort function.

# Create object to be stored in database

In [19]:
dbase_tools.prep_for_db(all_states)

dog_list = session.query(Dog)
for doggy in dog_list:
    print(doggy.name)

In [22]:
session_response = session.query(location_db.date, location_db.state_name, location_db.cases).\
    filter(location_db.state_name == "Illinois")

NameError: name 'location_db' is not defined

In [21]:
date_list